In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tqdm.notebook import tqdm
# from sklearn.model_selection import train_test_split
import jieba
import pickle
import re
import datetime

In [2]:
df = pd.read_csv("./DMSC.csv")

In [3]:
df.isna().sum()

ID                 0
Movie_Name_EN      0
Movie_Name_CN      0
Crawl_Date         0
Number             0
Username         149
Date               0
Star               0
Comment            0
Like               0
dtype: int64

In [4]:
sampled_df = df[["Comment", "Star"]]

In [ ]:
pd.value_counts(sampled_df["Star"]).plot.pie()

In [5]:
sampled_df = sampled_df.drop_duplicates(["Comment"], keep="first")


def process_star(row):
    if row["Star"] >= 4:
        return "1"
    else:
        return "0"


sampled_df["Star"] = sampled_df.apply(process_star, axis=1)


def strip_comment(row):
    return row["Comment"].strip()


sampled_df["Comment"] = sampled_df.apply(strip_comment, axis=1)

sampled_df.reset_index(drop=True, inplace=True)

sampled_df.head()

Comment Star
0                                      连奥创都知道整容要去韩国。    0
1  非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、妇...    0
2  2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出新...    0
3  《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于…...    1
4                                  虽然从头打到尾，但是真的很无聊啊。    0

In [8]:
pd.value_counts(sampled_df["Star"])

1    1179936
0     783088
Name: Star, dtype: int64

In [ ]:
comment = sampled_df["Comment"].values
star = sampled_df["Star"].values

# x_train, x_test, y_train, y_test = train_test_split(comment, star, test_size=0.2, random_state=0)

x_train_ori = comment
y_train_ori = star

with open("./crawler_x.txt", "r") as f:
    x_test_ori = f.readlines()
with open("./crawler_y.txt", "r") as f:
    y_test_ori = f.readlines()


def trans(x: int) -> int:
    if x >= 4:
        return 1
    else:
        return 0


y_test_ori = [trans(int(x)) for x in y_test_ori]

x_test_ori = np.array(x_test_ori)
y_test_ori = np.array(y_test_ori)

print(x_train_ori.shape, y_train_ori.shape)
print(x_test_ori.shape, y_test_ori.shape)

In [ ]:
with open("./stopwords.txt", "r", encoding="utf-8") as f:
    stopwords = f.readlines()


def process(comments, labels):
    assert len(comments) == len(labels)
    new_comments = []
    new_labels = []
    for idx, comment in enumerate(tqdm(comments)):
        comment = re.sub(r"[^\u4e00-\u9fff]", "", comment)
        comment = re.sub(r"[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", comment)
        comment = comment.strip()
        seg_comment = jieba.cut(comment)
        # seg_comment = filter(lambda x: len(x) > 1, seg_comment)
        seg_comment = list(filter(lambda x: x not in stopwords, seg_comment))
        if len(seg_comment):
            new_comments.append(seg_comment)
            new_labels.append(int(labels[idx]))
    return new_comments, new_labels


x_train_proc, y_train_proc = process(x_train_ori, y_train_ori)
x_test_proc, y_test_proc = process(x_test_ori, y_test_ori)

x_train = [" ".join(comment) for comment in x_train_proc]
x_test = [" ".join(comment) for comment in x_test_proc]
y_train = y_train_proc
y_test = y_test_proc

In [ ]:
with open("./x_train.txt", "w") as f:
    f.write("\n".join(x_train))
with open("./y_train.txt", "w") as f:
    f.write("\n".join(map(str, y_train)))
with open("./x_test.txt", "w") as f:
    f.write("\n".join(x_test))
with open("./y_test.txt", "w") as f:
    f.write("\n".join(map(str, y_test)))

------

In [2]:
with open("./x_train.txt", "r", encoding="utf-8") as f:
    x_train_str = f.readlines()
with open("./y_train.txt", "r", encoding="utf-8") as f:
    y_train_str = f.readlines()
with open("./x_test.txt", "r", encoding="utf-8") as f:
    x_test_str = f.readlines()
with open("./y_test.txt", "r", encoding="utf-8") as f:
    y_test_str = f.readlines()


def proc_y(y):
    res = [0, 0]
    res[int(y)] = 1
    return res


y_train_str = list(map(proc_y, y_train_str))
y_test_str = list(map(proc_y, y_test_str))

In [3]:
num_words = 2000
pad_len = 192
embed_dim = 128
lstm_out = 192

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=num_words, split=" ")

tokenizer.fit_on_texts(x_train_str)
tokenizer.fit_on_texts(x_test_str)

x_train_vec = tokenizer.texts_to_sequences(x_train_str)
x_test_vec = tokenizer.texts_to_sequences(x_test_str)

x_train = keras.preprocessing.sequence.pad_sequences(x_train_vec, maxlen=pad_len, dtype="float32")
x_test = keras.preprocessing.sequence.pad_sequences(x_test_vec, maxlen=pad_len, dtype="float32")

y_train = np.array(y_train_str)
y_test = np.array(y_test_str)

pickle.dump(tokenizer, open("./tokenizer.pkl", "wb"))

In [5]:
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [6]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(pad_len,)),
    keras.layers.Embedding(num_words, embed_dim),
    keras.layers.Dropout(0.5),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_out)),
    keras.layers.Dense(2, activation="sigmoid")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 192, 128)          256000    
_________________________________________________________________
dropout (Dropout)            (None, 192, 128)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 384)               493056    
_________________________________________________________________
dense (Dense)                (None, 2)                 770       
Total params: 749,826
Trainable params: 749,826
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x_train, y_train, batch_size=256, epochs=10, validation_split=0.1, callbacks=[tensorboard_callback])

model.evaluate(x_test, y_test)

model.save("./model.h5")

Epoch 1/10
6844/6844 [==============================] - 199s 29ms/step - loss: 0.4937 - accuracy: 0.7553 - val_loss: 0.3605 - val_accuracy: 0.8384
Epoch 2/10
6844/6844 [==============================] - 173s 25ms/step - loss: 0.4649 - accuracy: 0.7728 - val_loss: 0.3309 - val_accuracy: 0.8635
Epoch 3/10
6844/6844 [==============================] - 173s 25ms/step - loss: 0.4542 - accuracy: 0.7792 - val_loss: 0.3409 - val_accuracy: 0.8594
Epoch 4/10
6844/6844 [==============================] - 174s 25ms/step - loss: 0.4466 - accuracy: 0.7836 - val_loss: 0.3429 - val_accuracy: 0.8632
Epoch 5/10
6844/6844 [==============================] - 171s 25ms/step - loss: 0.4413 - accuracy: 0.7869 - val_loss: 0.3282 - val_accuracy: 0.8637
Epoch 6/10
6844/6844 [==============================] - 170s 25ms/step - loss: 0.4370 - accuracy: 0.7894 - val_loss: 0.3329 - val_accuracy: 0.8619
Epoch 7/10
6844/6844 [==============================] - 170s 25ms/step - loss: 0.4340 - accuracy: 0.7911 - val_loss: 0

In [7]:
model.load_weights("./model.h5")

In [7]:
with open("./test_result.txt", "w") as f:
    for i in tqdm(range(len(x_test_str))):
        f.write(x_test_str[i])
        neg, pos = model.predict(np.array([x_test[i]]))[0]
        f.write(f"label / predict: {'0' if y_test_str[i][0] == 1 else '1'} / {'0' if neg > pos else '1'}\n")
        f.write(f"neg / pos: {neg} / {pos}\n")
        f.write("\n")

  0%|          | 0/1773 [00:00<?, ?it/s]

In [9]:
with open("./wrong_result.txt", "w") as f:
    for i in tqdm(range(len(x_test_str))):
        neg, pos = model.predict(np.array([x_test[i]]))[0]
        if y_test_str[i][0] != (neg > pos):
            f.write(x_test_str[i])
            f.write(f"label / predict: {'0' if y_test_str[i][0] == 1 else '1'} / {'0' if neg > pos else '1'}\n")
            f.write(f"neg / pos: {neg} / {pos}\n")
            f.write("\n")

  0%|          | 0/1773 [00:00<?, ?it/s]